In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("result_seq2seq.csv")

In [3]:
#data["answer"] = data["answer"].str.replace('_', '')

data

In [4]:
ans = data["answer"]
pred = data["predict"]

In [5]:
ans

0           いらっしゃったうーん
1                十一時半に
2                   あっ
3                  十五分
4                  あはい
             ...      
21135            そうですね
21136             をですか
21137               うん
21138          あーあはいはい
21139    麻雀もおやりになるんですね
Name: answer, Length: 21140, dtype: object

# 完全一致率の計算

In [6]:
cnt = 0
for i in range(len(ans)):
    if ans[i] == pred[i]:
        #print("ans:"+ans[i]+" pred:"+pred[i])
        cnt+=1

In [7]:
cnt

272

In [8]:
same_percentage = (cnt/len(ans))*100

In [9]:
same_percentage

1.2866603595080417

# 種類数の計算

In [10]:
col = []
for i in range(len(pred)):
    if pred[i] not in col:
        #print(pred[i])
        col.append(pred[i])

In [11]:
len(col)

75

# BLUEの計算

入力に対する答えが複数あるのですべてに対してスコアを計算する


In [12]:
import nltk

In [13]:
df_ans = data.assign(bleu = 0)

In [14]:
df_ans

,Unnamed: 0,input,answer,predict,bleu
0,457,11時半頃,いらっしゃったうーん,あー,0
1,7866,11時半頃,十一時半に,あー,0
2,2131,15分ぐらいまで,あっ,あですです,0
3,6635,15分ぐらいまで,十五分,あ,0
4,4986,15分ぐらいまで駅に,あはい,そう,0
...,...,...,...,...,...
21135,696,鬼を抜けられないという遊びだったように,そうですね,あそう,0
21136,594,魚を,をですか,あー,0
21137,332,鳥を捕まえるという,うん,あー,0
21138,983,麻雀をするとか,あーあはいはい,あーなんですか,0


In [15]:
df_ans = df_ans.groupby("input").agg({
    'answer':list,
    "predict": list,
    "bleu" : list
})

In [16]:
df_ans = df_ans.reset_index()

In [17]:
df_ans

,input,answer,predict,bleu
0,11時半頃,"[いらっしゃったうーん, 十一時半に]","[あー, あー]","[0, 0]"
1,15分ぐらいまで,"[あっ, 十五分]","[あですです, あ]","[0, 0]"
2,15分ぐらいまで駅に,"[あはい, 15分はい]","[そう, うーん]","[0, 0]"
3,15分ぐらいまで駅に着くまでにかかりました,"[はいだったんですねありがとう, はいうーんうん, ええあそうですねそうだったんですね, あ...","[そう, あえ, あーんですか, あー]","[0, 0, 0, 0]"
4,15分前に,"[や, 凄いです, あっ, 通りああ]","[あー, あーなんですか, へえ, あそうなんですか]","[0, 0, 0, 0]"
...,...,...,...,...
7367,鳥を捕まえるという,"[ええええ, 捕まえるんですか, へー, えー, おー, うん]","[あ, あそうなんですか, あーです, あそうですです, あ, あー]","[0, 0, 0, 0, 0, 0]"
7368,鳥を捕まえるというそんな,"[へえ, 捕まえるんですね, あはいそうですね, 鳥をですか, 鳥, 鳥を, へえ]","[あーんですか, あーんですです, はいありがとうございます嬉しかっふ, あ, あそうですで...","[0, 0, 0, 0, 0, 0, 0]"
7369,麹町番長の,[そうなんですね],[あーです],[0]
7370,麹町番長の大きなお屋敷のおー石造りのバルコニーみたいな部分だけ残っているとか,"[そうなんですか, ああ, あー, はー]","[あ, あはいです, そう, あです]","[0, 0, 0, 0]"


In [18]:
from janome.tokenizer import Tokenizer
j_t = Tokenizer()

def tokenizer(text):
    return [tok for tok in j_t.tokenize(text, wakati = True)]

In [19]:
len(df_ans)

7372

In [20]:
df_ans.iloc[3027]

input               でえー
answer     [うんうん, すごーい]
predict         [そう, あ]
bleu             [0, 0]
Name: 3027, dtype: object

In [21]:
col, ans, pred = [], [], []
for k in range(len(df_ans)):
    for j in range(len(df_ans["predict"][k])):
        
        for i in range(len(df_ans["answer"][k])):
            ans_str = df_ans["answer"][k][i]
            ans.append(tokenizer(ans_str))
        
        print(k)
        pred = df_ans["predict"][k][j]
        #print(pred)
        pred = tokenizer(pred)
        col.append(nltk.translate.bleu_score.sentence_bleu(ans, pred))
        ave = sum(col)/len(col)
        #print(ave)
        df_ans["bleu"][k][j] = ave
        col, ans = [], []
    

0
0
1
1
2
2
3
3
3
3
4
4
4
4
5
5
6
6
6
6
6
6
6
6
7
7
7
8
8
9
9
9
9
9
10
10
11
12
12
12
12
12
12
12
13
14
15
15
15
16


C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisc\anaconda3\envs\study\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

17
17
17
17
17
18
18
18
19
19
20
20
20
21
22
23
23
23
23
24
24
25
25
26
27
27
27
28
28
28
29
30
31
32
32
33
33
34
34
34
34
35
36
36
37
37
37
37
37
37
37
37
38
39
40
41
42
42
42
42
43
44
45
45
45
45
45
45
45
45
46
46
46
47
47
48
49
50
50
51
52
53
53
53
53
53
53
53
53
53
53
53
54
55
55
55
56
56
57
57
57
57
57
58
58
59
60
61
61
62
62
62
62
63
63
63
64
65
65
65
66
67
67
67
67
67
67
68
69
69
69
69
69
70
70
71
71
72
72
72
72
73
73
73
73
73
73
73
73
73
73
73
74
75
76
77
77
77
77
77
78
78
78
78
78
78
78
78
79
80
80
80
81
82


AttributeError: 'float' object has no attribute 'strip'

In [22]:
df_ans

,input,answer,predict,bleu
0,11時半頃,"[いらっしゃったうーん, 十一時半に]","[あー, あー]","[0.0, 0.0]"
1,15分ぐらいまで,"[あっ, 十五分]","[あですです, あ]","[0.0, 0.0]"
2,15分ぐらいまで駅に,"[あはい, 15分はい]","[そう, うーん]","[0.0, 0.0]"
3,15分ぐらいまで駅に着くまでにかかりました,"[はいだったんですねありがとう, はいうーんうん, ええあそうですねそうだったんですね, あ...","[そう, あえ, あーんですか, あー]","[2.4655814830110698e-232, 1.1049969584438628e-..."
4,15分前に,"[や, 凄いです, あっ, 通りああ]","[あー, あーなんですか, へえ, あそうなんですか]","[0.0, 1.2183324802375697e-231, 0.0, 1.16404698..."
...,...,...,...,...
7367,鳥を捕まえるという,"[ええええ, 捕まえるんですか, へー, えー, おー, うん]","[あ, あそうなんですか, あーです, あそうですです, あ, あー]","[0, 0, 0, 0, 0, 0]"
7368,鳥を捕まえるというそんな,"[へえ, 捕まえるんですね, あはいそうですね, 鳥をですか, 鳥, 鳥を, へえ]","[あーんですか, あーんですです, はいありがとうございます嬉しかっふ, あ, あそうですで...","[0, 0, 0, 0, 0, 0, 0]"
7369,麹町番長の,[そうなんですね],[あーです],[0]
7370,麹町番長の大きなお屋敷のおー石造りのバルコニーみたいな部分だけ残っているとか,"[そうなんですか, ああ, あー, はー]","[あ, あはいです, そう, あです]","[0, 0, 0, 0]"


In [23]:
input_str, answer, predict, bleu = [], [], [], []
for i in range(len(df_ans)):
    for j in range(len(df_ans["answer"][i])):
        input_str.append(df_ans["input"][i])
        answer.append(df_ans["answer"][i][j])
        predict.append(df_ans["predict"][i][j])
        bleu.append(df_ans["bleu"][i][j])

In [24]:
input_str

['11時半頃',
 '11時半頃',
 '15分ぐらいまで',
 '15分ぐらいまで',
 '15分ぐらいまで駅に',
 '15分ぐらいまで駅に',
 '15分ぐらいまで駅に着くまでにかかりました',
 '15分ぐらいまで駅に着くまでにかかりました',
 '15分ぐらいまで駅に着くまでにかかりました',
 '15分ぐらいまで駅に着くまでにかかりました',
 '15分前に',
 '15分前に',
 '15分前に',
 '15分前に',
 '15分前に事務所に',
 '15分前に事務所に',
 '15分前に事務所に着きました',
 '15分前に事務所に着きました',
 '15分前に事務所に着きました',
 '15分前に事務所に着きました',
 '15分前に事務所に着きました',
 '15分前に事務所に着きました',
 '15分前に事務所に着きました',
 '15分前に事務所に着きました',
 '16年間その先生にお世話になって',
 '16年間その先生にお世話になって',
 '16年間その先生にお世話になって',
 '18歳以上でないとあのやる',
 '18歳以上でないとあのやる',
 '18歳以上でないとあのやることができ',
 '18歳以上でないとあのやることができ',
 '18歳以上でないとあのやることができ',
 '18歳以上でないとあのやることができ',
 '18歳以上でないとあのやることができ',
 '1964年昭和',
 '1964年昭和',
 '1センチ5ミリぐらいの',
 '1回も行っておりませんので',
 '1回も行っておりませんので',
 '1回も行っておりませんので',
 '1回も行っておりませんので',
 '1回も行っておりませんので',
 '1回も行っておりませんので',
 '1回も行っておりませんので',
 '1回も行っておりませんのでもう',
 '1回も行っておりませんのでもうここ',
 '1日中家にいながらも',
 '1日中家にいながらも',
 '1日中家にいながらも',
 '1日中家にいながらもあの',
 '1日中家にいながらもあのやっておりまして',
 '1日中家にいながらもあのやっておりまして',
 '1日中家にいながらもあのやっておりまして',
 '1日中家にいながらもあのやっておりまして',
 '1日中家にいながらもあのやっ

In [25]:
df = pd.DataFrame({"input":input_str,"answer":answer,"predict":predict,"bleu":bleu})

In [26]:
df

,input,answer,predict,bleu
0,11時半頃,いらっしゃったうーん,あー,0.0
1,11時半頃,十一時半に,あー,0.0
2,15分ぐらいまで,あっ,あですです,0.0
3,15分ぐらいまで,十五分,あ,0.0
4,15分ぐらいまで駅に,あはい,そう,0.0
...,...,...,...,...
21135,麻雀をするとか,あはい,あ,0.0
21136,麻雀をするとか,あーおおおお,あー,0.0
21137,麻雀をするとか,あー麻雀も,あはい,0.0
21138,麻雀をするとか,あーあはいはい,あーなんですか,0.0


In [27]:
df.to_csv('./score_seq2seq.csv')

In [28]:
blue = df["bleu"]

In [29]:
type(blue)

pandas.core.series.Series

In [30]:
l = blue.values.tolist()

In [31]:
type(l)

list

In [32]:
ave_blue = sum(l)/len(l)

In [33]:
ave_blue

0.00013724672332236126